### 이번주 과제는 DACON의 구내 식당 식수인원 예측 AI 경진대회의 데이터를 그대로 사용하여 직접 모델링 하고 제출까지 해보는 과제입니다! 

Decision Tree의 파라미터 튜닝 외에도 이전 교육세션에서 배우신 Feature-engineering, 다른 회귀 모델도 자율적으로 사용하시는걸 모두 허용합니다 :) 처음 하시는 분들도 아 이런식으로 데이터 공모전에 참여하는구나의 감만 익혀가시는 걸 목표로 하시면 좋을 것 같습니다. 과제하시면서 질문이 있으신 분들은 언제든 질문방 or 개인톡으로 연락주세요 :)

from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pandas as pd
import numpy as np

### 데이터셋 로드

In [2]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
submission= pd.read_csv('sample_submission.csv')

In [3]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [4]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [5]:
submission.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


### 요일을 숫자로 매핑

In [6]:
train.요일.unique()

array(['월', '화', '수', '목', '금'], dtype=object)

In [7]:
train['요일']= train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
test['요일']= test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})

In [8]:
train['요일'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

### 수치 데이터만 feature로 선택

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   일자              1205 non-null   object 
 1   요일              1205 non-null   int64  
 2   본사정원수           1205 non-null   int64  
 3   본사휴가자수          1205 non-null   int64  
 4   본사출장자수          1205 non-null   int64  
 5   본사시간외근무명령서승인건수  1205 non-null   int64  
 6   현본사소속재택근무자수     1205 non-null   float64
 7   조식메뉴            1205 non-null   object 
 8   중식메뉴            1205 non-null   object 
 9   석식메뉴            1205 non-null   object 
 10  중식계             1205 non-null   float64
 11  석식계             1205 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 113.1+ KB


In [10]:
## df[[a, b, c]] --> rearrange도 되고 아니면 저것만 뽑는것도 되고

x_train = train[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]

y1_train = train['중식계']
y2_train = train['석식계']

x_test = test[['요일', '본사정원수', '본사출장자수', '본사시간외근무명령서승인건수', '현본사소속재택근무자수']]

### Decision tree model 생성

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

### 석식계 예측모델과 중식계 예측모델 Train test split

In [12]:
x_train_lunch, x_val_lunch, y_train_lunch, y_val_lunch = train_test_split(x_train, y1_train, test_size=0.2, random_state=42)
x_train_dinner, x_val_dinner, y_train_dinner, y_val_dinner = train_test_split(x_train, y2_train, test_size=0.2, random_state=42)

Q) validation set의 역할은 무엇인가요?

A) validation set의 역할은 학습이 이미 완료된 모델을 검증하기 위함 입니다.

### 중식계 예측모델 Grid Search
- parameter의 값을 다양하게 해서 model selection 하기

In [15]:
from sklearn.model_selection import GridSearchCV
grid_dt = DecisionTreeRegressor()

#테스트하고 싶은 parameter 값들 정하기

max_depth= [2, 3, 4, 5, 6, 7]
max_leaf=list(np.arange(4,30,2))

#하나로 묶기 위해 dictionary 형태로 만들기

parameter_grid={'max_depth': max_depth,
               'max_leaf_nodes' : max_leaf}

In [16]:
gs = GridSearchCV(estimator=grid_dt, param_grid= parameter_grid)

In [17]:
gs.fit(x_train_lunch, y_train_lunch)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7],
                         'max_leaf_nodes': [4, 6, 8, 10, 12, 14, 16, 18, 20, 22,
             

In [20]:
print('GridSearch 최적 parameter: {}'.format(gs.best_params_),
     'GridSearch 최고 Validation Score: {:.3f}'.format(gs.best_score_), sep='\n')

GridSearch 최적 parameter: {'max_depth': 6, 'max_leaf_nodes': 18}
GridSearch 최고 Validation Score: 0.677


### 중식계 예측모델 학습
- 위 과정(Grid Search)로 찾은 최적의 parameter 값으로 모델 생성하기

In [23]:
#Grid Search로 찾은 최적의 parameter 값으로 모델 생성하기

from sklearn.tree import DecisionTreeRegressor
model1 = DecisionTreeRegressor(max_depth=6, max_leaf_nodes=18)

# model fit 이후 validation dataset으로 predict 해보기

model1.fit(x_train_lunch, y_train_lunch)
reg_prediction = model1.predict(x_val_lunch)

In [26]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

print('MAE : {:.3f}'.format(mean_absolute_error(y_val_lunch, reg_prediction)))
print('MSE : {:.3f}'. format(mean_squared_error(y_val_lunch, reg_prediction)))
print('r2 score : {:.3f}'.format(r2_score(y_val_lunch, reg_prediction)))

MAE : 94.741
MSE : 16608.659
r2 score : 0.608


### 학습된 모델로 중식계 test data 예측

Q) MAE, MSE, r2_score가 각각 의미하는 바를 간단하게 적어주세요.

A) 이들은 모두 회귀 모델을 평가하는 지표에 해당한다. MAE는 mean absolute error의 준 말로, 오차에 대한 절댓값의 평균을 의미한다. MSE는 mean squared error의 준말로, 오차에 대한 제곱의 평균을 의미한다. r2_score은 전체 변동량에 대하여, 회귀식에 의해 설명되는 변동량의 비를 나타낸다.

In [27]:
pred1= model1.predict(x_test)

In [28]:
submission['중식계'] = pred1
submission.head()

,일자,중식계,석식계
0,2021-01-27,929.437956,0
1,2021-01-28,929.437956,0
2,2021-01-29,598.504505,0
3,2021-02-01,1178.644444,0
4,2021-02-02,929.437956,0


### 석식계 예측모델 Grid Search

In [29]:
gs.fit(x_train_dinner, y_train_dinner)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse',
                                             max_depth=None, max_features=None,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             presort='deprecated',
                                             random_state=None,
                                             splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7],
                         'max_leaf_nodes': [4, 6, 8, 10, 12, 14, 16, 18, 20, 22,
             

In [33]:
print('GridSearch 최적 parameter: {}'.format(gs.best_params_),
     'GridSearch 최고 Validation Score: {:.3f}'.format(gs.best_score_), sep='\n')

GridSearch 최적 parameter: {'max_depth': 4, 'max_leaf_nodes': 18}
GridSearch 최고 Validation Score: 0.343


### 석식계 예측모델 학습

In [34]:
# Grid Search 로 찾은 최적의 parameter 값으로 모델 생성하기

model2= DecisionTreeRegressor(max_depth=4, max_leaf_nodes=18)

# 모델 적합하고, validation dataset으로 predict

model2.fit(x_train_dinner, y_train_dinner)
reg_prediction2 = model2.predict(x_val_dinner)

In [36]:
print('MAE: {:.3f}'.format(mean_absolute_error(y_val_dinner, reg_prediction2)))
print('MSE: {:.3f}'.format(mean_squared_error(y_val_dinner, reg_prediction2)))
print('r2 score: {:.3f}'.format(r2_score(y_val_dinner, reg_prediction2)))

MAE: 78.808
MSE: 12974.235
r2 score: 0.346


### 학습된 모델로 석식계 test data 예측

In [37]:
pred2= model2.predict(x_test)

In [38]:
submission['석식계']=pred2
submission.head()

,일자,중식계,석식계
0,2021-01-27,929.437956,333.788288
1,2021-01-28,929.437956,414.317073
2,2021-01-29,598.504505,333.788288
3,2021-02-01,1178.644444,414.317073
4,2021-02-02,929.437956,414.317073


### submission.csv 저장 후 제출

In [39]:
submission.to_csv('dt_submission.csv', index=False)

#### 본인의 깃헙 repository에 과제 코드와 제출한 화면 캡처본을 함께 업로드해주세요 